# Script for reading Serial port to get sensors values 

Change the serial port to the good one (Devices Manager on Windows).  
Restart whole kernel each time.

In [1]:
import serial
import time 
import pandas as pd
import numpy as np
import os
from IPython.display import clear_output

ESP_serial = serial.Serial('COM6',9600)

header = ["IR0_meas", "IR1_meas", "IR2_meas", "IR3_meas"]
path = "./content/measures-example.csv"

# meas_data = np.random.rand(1,4)
# meas_df = pd.DataFrame(meas_data,columns = header)
# meas_df.to_csv(path,index=False, mode='a', header=not os.path.exists(path))

mean_buff = []


i = 0
mean_nbr = 5


try:
    while True :

        data_read = ESP_serial.readline().decode('utf-8')[:-1]
        print("\rReceiving data...")
        split = data_read.split(",")
        meas_data = []
        for each in split :
            meas_data.append(float(each))
        mean_buff.append(meas_data)

        i=i+1
        if i >= mean_nbr :

            row = np.mean(np.array(mean_buff), axis=0)
            # save to csv
            meas_df = pd.DataFrame(np.array([row]),columns = header)
            meas_df.to_csv(path,index=False, mode='w')

            clear_output(wait=True)
            print("\n Meaning buffer : ")   
            print(np.array(mean_buff))
            print("\n Average distances written to : ", path)
            print(meas_df)
            print("\n")
            
            # reset variables
            mean_buff = []
            i = 0

        time.sleep(0.1)
except KeyboardInterrupt:
    ESP_serial.close()
    print("interrupt")
    pass